In [1]:
# !git clone https://github.com/bazingagin/npc_gzip.git

In [2]:
!pip install npc-gzip datasets portalocker>=2.0.0 ipython-autotime scikit-learn -q

In [3]:
import numpy as np
from sklearn.metrics import classification_report
from torchtext.datasets import IMDB

from npc_gzip.compressors.base import BaseCompressor
from npc_gzip.compressors.gzip_compressor import GZipCompressor
from npc_gzip.knn_classifier import KnnClassifier
import pandas as pd
from tqdm import tqdm, tqdm_pandas
%load_ext autotime
import random
import numpy as np
np.random.seed(seed=21)

time: 441 µs (started: 2023-08-20 12:25:03 +00:00)


In [4]:
!pip install nltk -q
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

time: 8.09 s (started: 2023-08-20 12:25:03 +00:00)


In [5]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    cleaned_text = cleaned_text.lower()
    cleaned_text = ' '.join(cleaned_text.split())
    words = cleaned_text.split()
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

time: 5.83 ms (started: 2023-08-20 12:25:11 +00:00)


In [6]:
# Example usage
input_text = "This is an example text with some extra    spaces and irrelevant words."
cleaned_output = clean_text(input_text)
print(cleaned_output)

example text extra space irrelevant word
time: 2.37 s (started: 2023-08-20 12:25:11 +00:00)


In [7]:
from datasets import load_dataset
data = load_dataset('imdb')
data['train'] = data['train'].shuffle(seed=42)
data['test'] = data['test'].shuffle(seed=42)

time: 1.04 s (started: 2023-08-20 12:25:14 +00:00)


In [10]:
# conveting to pandas
train = data['train'].to_pandas()
test  = data['test'].to_pandas()
print(train.columns)
## cleaning text
train['clean_text'] = train['text'].apply(lambda x:clean_text(x))
test['clean_text'] = test['text'].apply(lambda x:clean_text(x))


Index(['text', 'label'], dtype='object')
time: 37 s (started: 2023-08-20 12:27:07 +00:00)


## NPC_GZIP Model

In [11]:
#converting to numpy
train_cl_text,train_cl_labels = train['clean_text'].values,train['label'].values
test_cl_text,test_cl_labels = test['clean_text'].values,test['label'].values
print(f"Training dataset size {train_cl_text.shape}")
print(f"Testin dataset size {test_cl_text.shape}")

Training dataset size (25000,)
Testin dataset size (25000,)
time: 3.18 ms (started: 2023-08-20 12:27:44 +00:00)


In [12]:
def fit_model(
    train_text: np.ndarray, train_labels: np.ndarray, distance_metric: str = "ncd"
) -> KnnClassifier:
    compressor: BaseCompressor = GZipCompressor()
    model: KnnClassifier = KnnClassifier(
        compressor=compressor,
        training_inputs=train_text,
        training_labels=train_labels,
        distance_metric=distance_metric,
    )
    return model

time: 803 µs (started: 2023-08-20 12:27:44 +00:00)


In [16]:
def main(train_text, train_labels,test_text,test_labels) -> None:
    print("Fitting model...")
    model = fit_model(train_text, train_labels)
    print("Generating predictions...")
    top_k = 1
    (distances, labels, similar_samples) = model.predict(
        test_text, top_k, sampling_percentage=0.01
    )
    print(classification_report(test_labels, labels.reshape(-1)))

time: 913 µs (started: 2023-08-20 12:28:09 +00:00)


### Model on text data

In [17]:
#converting to numpy
train_text,train_labels = train['text'].values,train['label'].values
test_text,test_labels = test['text'].values,test['label'].values
print(f"Training dataset size {train_text.shape}")
print(f"Testin dataset size {test_text.shape}")

Training dataset size (25000,)
Testin dataset size (25000,)
time: 3.79 ms (started: 2023-08-20 12:28:18 +00:00)


In [18]:
main(train_text, train_labels,test_text,test_labels)

Fitting model...
Generating predictions...


Compressing input...: 100%|██████████| 25000/25000 [13:39<00:00, 30.49it/s]


              precision    recall  f1-score   support

           0       0.56      0.65      0.60     12500
           1       0.59      0.49      0.54     12500

    accuracy                           0.57     25000
   macro avg       0.57      0.57      0.57     25000
weighted avg       0.57      0.57      0.57     25000

time: 13min 46s (started: 2023-08-20 12:28:24 +00:00)


### Model on clean text data

In [19]:
#converting to numpy
train_cl_text,train_cl_labels = train['clean_text'].values,train['label'].values
test_cl_text,test_cl_labels = test['clean_text'].values,test['label'].values
print(f"Training dataset size {train_cl_text.shape}")
print(f"Testin dataset size {test_cl_text.shape}")

Training dataset size (25000,)
Testin dataset size (25000,)
time: 3.73 ms (started: 2023-08-20 12:45:33 +00:00)


In [20]:
main(train_cl_text, train_cl_labels,test_cl_text,test_cl_labels)

Fitting model...
Generating predictions...


Compressing input...: 100%|██████████| 25000/25000 [08:51<00:00, 47.01it/s]


              precision    recall  f1-score   support

           0       0.59      0.55      0.57     12500
           1       0.57      0.61      0.59     12500

    accuracy                           0.58     25000
   macro avg       0.58      0.58      0.58     25000
weighted avg       0.58      0.58      0.58     25000

time: 8min 59s (started: 2023-08-20 12:45:36 +00:00)


## Random Forest with TFIDF

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

def train_and_evaluate_random_forest(train_text, train_labels, test_text, test_labels):
    # Initialize TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(max_features=5000)
    # Transform the training text data to TF-IDF features
    train_tfidf = tfidf_vectorizer.fit_transform(train_text)

    # Transform the test text data using the same TF-IDF vectorizer
    test_tfidf = tfidf_vectorizer.transform(test_text)

    # Initialize the Random Forest classifier
    rf_classifier = RandomForestClassifier(n_estimators=100,n_jobs=-1,random_state=42)

    # Train the model
    rf_classifier.fit(train_tfidf, train_labels)

    # Make predictions on the test data
    predictions = rf_classifier.predict(test_tfidf)

    # Calculate accuracy
    accuracy = accuracy_score(test_labels, predictions)

    # Print the classification report
    class_report = classification_report(test_labels, predictions)
    print(class_report)


train_and_evaluate_random_forest(train_cl_text, train_cl_labels,test_cl_text,test_cl_labels)

              precision    recall  f1-score   support

           0       0.83      0.86      0.84     12500
           1       0.85      0.82      0.84     12500

    accuracy                           0.84     25000
   macro avg       0.84      0.84      0.84     25000
weighted avg       0.84      0.84      0.84     25000

time: 54.4 s (started: 2023-08-20 13:00:35 +00:00)
